In [56]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd



In [57]:
#Target web page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


In [59]:
#Connection to web page and converting response into python string
response = requests.get(url).text

Create a beautiful soup object to parse the HTML document.

This step helps one to access the elements of the web site by using XPath expressions

In [60]:
soup = BeautifulSoup(response, 'xml')
table=soup.find('table')






In [61]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

Finding all the items of  interest with .find_all()
This command allows you to specify and find all the elements you are searching for. 


In [62]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [63]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [64]:
#Data Cleaning
#Remove rows where Borough is not assigned
df=df[df.Borough!='Not assigned']

Grouping different neighbors and seperating them with a comma.

In [67]:
res_df=df.groupby("Postalcode").agg(lambda x:','.join(set(x)))

In [68]:
#Assigning an empty neighborhood the next one
res_df.loc[res_df['Neighborhood']=="Not assigned",'Neighborhood']=res_df.loc[temp_df['Neighborhood']=="Not assigned",'Borough']

In [69]:
res_df.head()

,Borough,Neighborhood
Postalcode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [70]:
res_df.shape

(103, 2)

Importing relevant libraries

In [77]:
import numpy as np 
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Download the csv file with coordinates and assign it to a dataframe.
Rename Postal Code column to allow merging

In [81]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data   
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
# Rename the Postal Code column to allow merging
df_coord.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
print(df_coord[0:3])

  Postalcode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711


In [85]:
# Merge the  dataframes on the Postalcode column\n",
new_df = pd.merge(res_df, df_coord, on='Postalcode' )
new_df.head(20)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
